In [ ]:
# Install required libraries
import os

try:
    import selenium
except ImportError:
    os.system("pip install selenium")

try:
    import webdriver_manager
except ImportError:
    os.system("pip install webdriver-manager")

try:
    import pandas as pd
except ImportError:
    os.system("pip install pandas")

In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import time

In [2]:
# Initialize Chrome with options
chrome_options = Options()
chrome_options.add_argument("--start-maximized")  # Start maximized for better visibility
chrome_options.add_argument("--disable-infobars")  # Disable infobars
chrome_options.add_argument("--disable-extensions")  # Disable extensions for simplicity
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)

def scroll_to_bottom(driver, pause_time=2):
    """
    Scroll to the bottom of the page to load dynamic content.
    """
    last_height = driver.execute_script("return document.body.scrollHeight")

    while True:
        # Scroll down to bottom
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

        # Wait to load page
        time.sleep(pause_time)

        # Calculate new scroll height and compare with last scroll height
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height

driver.get("https://www.expedia.com/Hotel-Search?adults=2&children=&d1=2024-12-13&d2=2024-12-18&destination=Bangkok%20%28and%20vicinity%29%2C%20Bangkok%20Province%2C%20Thailand&endDate=2024-12-19&latLong=13.7475%2C100.53601&lodging=HOTEL&mapBounds=&pwaDialog=&regionId=178236&rooms=1&semdtl=&sort=PROPERTY_CLASS&startDate=2024-12-14&theme=&useRewards=false&userIntent=")
time.sleep(6)  # Wait for the page to load completely

# Extract hotel data
listing_container = WebDriverWait(driver, 20).until(
    EC.presence_of_element_located((By.CSS_SELECTOR, 'div[data-stid="property-listing-results"]'))
)
hotel_links = listing_container.find_elements(By.CSS_SELECTOR, 'a[data-stid="open-hotel-information"]')
print(f"Found {len(hotel_links)} hotel links.")


Found 303 hotel links.


In [ ]:
import pandas as pd

# Initialize a list to store all hotel data
hotel_data = []
location = "Bangkok, Thailand"

for index, link in enumerate(hotel_links):
    try:
        href = link.get_attribute('href')
        print(f"\nProcessing link {index + 1}")

        # Open the link in a new tab using JavaScript
        driver.execute_script("window.open(arguments[0]);", href)
        time.sleep(3)  # Wait for the new tab to open

        # Switch to the new tab
        driver.switch_to.window(driver.window_handles[-1])
        time.sleep(15)  # Wait for the new page to load

        hotel_details = {"Location": location}

        # Extract Hotel Name
        try:
            # First, try to locate the hotel name with the old structure
            try:
                hotel_name_element = WebDriverWait(driver, 20).until(
                    EC.presence_of_element_located((By.CSS_SELECTOR, "h1.uitk-heading.uitk-heading-3"))
                )
                hotel_details['Hotel Name'] = hotel_name_element.text.strip()
            except Exception:
                # If the old structure fails, try the new structure
                hotel_name_element = WebDriverWait(driver, 20).until(
                    EC.presence_of_element_located((By.CSS_SELECTOR, "h1.uitk-heading.uitk-heading-4"))
                )
                hotel_details['Hotel Name'] = hotel_name_element.text.strip()

            print(f"Extracted Hotel Name: {hotel_details['Hotel Name']}")
        except Exception as e:
            print(f"Error extracting hotel name: {e}")
            hotel_details['Hotel Name'] = None

        # Extract Stars
        try:
            stars_container = driver.find_element(By.CSS_SELECTOR, "div.uitk-rating.uitk-spacing.uitk-spacing-margin-blockstart-one")
            stars_count = len(stars_container.find_elements(By.TAG_NAME, "svg"))
            hotel_details['Stars'] = stars_count
        except Exception as e:
            print(f"Error extracting stars for hotel {index + 1}: {e}")
            hotel_details['Stars'] = None

        # Extract Rating
        try:
            rating_element = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located(
                    (By.CSS_SELECTOR, "span.uitk-badge-base-large.uitk-badge-positive > span.uitk-badge-base-text")
                )
            )
            hotel_details['Rating'] = rating_element.text.strip()
        except Exception as e:
            print(f"Error extracting rating: {e}")
            hotel_details['Rating'] = None

        # Extract Review Keyword
        try:
            review_summary_container = WebDriverWait(driver, 20).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, "div[data-stid='content-hotel-reviewsummary']"))
            )
            review_keyword_element = review_summary_container.find_element(
                By.CSS_SELECTOR, "div.uitk-text.uitk-type-500.uitk-type-medium.uitk-text-default-theme"
            )
            review_keyword = review_keyword_element.text.strip()
            hotel_details['Review Keyword'] = review_keyword
            print(f"Extracted Review Keyword: {review_keyword}")
        except Exception as e:
            print(f"Error extracting review keyword: {e}")
            hotel_details['Review Keyword'] = None

        # Extract Services
        try:
            services_list = WebDriverWait(driver, 20).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, "ul.uitk-typelist[data-stid='sp-content-list']"))
            )

            services_lis = services_list.find_elements(By.CSS_SELECTOR, "li[data-stid^='sp-content-item-']")
            service_count = len(services_lis)
            print(f"Number of services: {service_count}")

            services = []
            for i in range(service_count):
                try:
                    service_span = driver.find_element(
                        By.CSS_SELECTOR,
                        f"ul.uitk-typelist[data-stid='sp-content-list'] li[data-stid='sp-content-item-{i}'] span.uitk-text.uitk-type-300.uitk-text-default-theme.uitk-layout-flex-item"
                    )
                    services.append(service_span.text.strip())
                except Exception as e:
                    print(f"Error extracting service {i}: {e}")
                    services.append(None)

            hotel_details['Services'] = services
        except Exception as e:
            print(f"Error extracting services: {e}")
            hotel_details['Services'] = []

        # Extract Property Offers
        try:
            property_offers_container = WebDriverWait(driver, 20).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, 'div[data-stid="section-room-list"]'))
            )

            property_offers = property_offers_container.find_elements(By.CSS_SELECTOR, 'div[data-stid^="property-offer-"]')

            offers = []
            for offer_index, offer in enumerate(property_offers, start=1):
                offer_details = {}

                try:
                    room_name_element = offer.find_element(By.CSS_SELECTOR, "h3.uitk-heading.uitk-heading-6")
                    offer_details['Room Name'] = room_name_element.text.strip()
                except Exception as e:
                    print(f"Error extracting room name for offer {offer_index}: {e}")
                    offer_details['Room Name'] = None

                try:
                    price_summary_div = offer.find_element(By.CSS_SELECTOR, 'div[data-stid="price-summary"]')
                    room_charges_element = price_summary_div.find_element(
                        By.CSS_SELECTOR,
                        "div.uitk-text.uitk-type-500.uitk-type-medium.uitk-text-emphasis-theme"
                    )
                    offer_details['Room Charges'] = room_charges_element.text.strip()
                except Exception as e:
                    print(f"Error extracting room charges for offer {offer_index}: {e}")
                    offer_details['Room Charges'] = None

                offers.append(offer_details)

            hotel_details['Property Offers'] = offers
        except Exception as e:
            print(f"Error extracting property offers: {e}")
            hotel_details['Property Offers'] = []

        # Add hotel details to list
        hotel_data.append(hotel_details)
        print(f"Extracted Hotel Data: {hotel_details}")

        # Close the new tab
        driver.close()

        # Switch back to the main tab
        driver.switch_to.window(driver.window_handles[0])
        time.sleep(1)

    except Exception as e:
        print(f"Error processing link {index + 1}: {e}")
        continue

# Export hotel data to a CSV file
df = pd.DataFrame(hotel_data)

# Normalize property offers if necessary
if 'Property Offers' in df.columns:
    df['Property Offers'] = df['Property Offers'].apply(lambda x: ', '.join(str(offer) for offer in x) if isinstance(x, list) else x)

# Save to CSV
output_filename = f"hotel_data_{location.replace(',', '').replace(' ', '_')}.csv"
df.to_csv(output_filename, index=False)
print(f"Data exported to {output_filename}")


Processing link 1
Extracted Hotel Name: Divalux Resort & Spa Bangkok, Suvarnabhumi Airport
Extracted Review Keyword: Very good
Number of services: 6
Extracted Hotel Data: {'Location': 'Bangkok, Thailand', 'Hotel Name': 'Divalux Resort & Spa Bangkok, Suvarnabhumi Airport', 'Stars': 5, 'Rating': '8.2', 'Review Keyword': 'Very good', 'Services': ['Pool', 'Free airport shuttle', 'Spa', 'Gym', 'Laundry', 'Breakfast available'], 'Property Offers': []}

Processing link 2
Extracted Hotel Name: The Landmark Bangkok
Extracted Review Keyword: Wonderful
Number of services: 6
Extracted Hotel Data: {'Location': 'Bangkok, Thailand', 'Hotel Name': 'The Landmark Bangkok', 'Stars': 5, 'Rating': '9.2', 'Review Keyword': 'Wonderful', 'Services': ['Pool', 'Breakfast available', 'Restaurant', 'Gym', 'Spa', 'Bar'], 'Property Offers': []}

Processing link 3
Extracted Hotel Name: lebua at State Tower
Extracted Review Keyword: Wonderful
Number of services: 6
Extracted Hotel Data: {'Location': 'Bangkok, Thailan


Processing link 22
Extracted Hotel Name: Valia Hotel Bangkok Sukhumvit
Extracted Review Keyword: Wonderful
Number of services: 6
Extracted Hotel Data: {'Location': 'Bangkok, Thailand', 'Hotel Name': 'Valia Hotel Bangkok Sukhumvit', 'Stars': 5, 'Rating': '9.0', 'Review Keyword': 'Wonderful', 'Services': ['Breakfast available', 'Gym', 'Pool', 'Restaurant', 'Bar', 'Parking available'], 'Property Offers': []}

Processing link 23
Extracted Hotel Name: Centara Grand at CentralWorld
Extracted Review Keyword: Wonderful
Number of services: 6
Extracted Hotel Data: {'Location': 'Bangkok, Thailand', 'Hotel Name': 'Centara Grand at CentralWorld', 'Stars': 5, 'Rating': '9.0', 'Review Keyword': 'Wonderful', 'Services': ['Breakfast available', 'Restaurant', 'Pool', 'Bar', 'Room service', 'Gym'], 'Property Offers': []}

Processing link 24
Extracted Hotel Name: Evergreen Laurel Hotel Bangkok
Extracted Review Keyword: Excellent
Number of services: 6
Extracted Hotel Data: {'Location': 'Bangkok, Thailand'


Processing link 43
Extracted Hotel Name: Tower Club at lebua
Extracted Review Keyword: Wonderful
Number of services: 6
Extracted Hotel Data: {'Location': 'Bangkok, Thailand', 'Hotel Name': 'Tower Club at lebua', 'Stars': 5, 'Rating': '9.0', 'Review Keyword': 'Wonderful', 'Services': ['Bar', 'Pool', 'Breakfast included', 'Room service', 'Gym', 'Laundry'], 'Property Offers': []}

Processing link 44
Extracted Hotel Name: Swissotel Bangkok Ratchada
Extracted Review Keyword: Excellent
Number of services: 6
Extracted Hotel Data: {'Location': 'Bangkok, Thailand', 'Hotel Name': 'Swissotel Bangkok Ratchada', 'Stars': 5, 'Rating': '8.8', 'Review Keyword': 'Excellent', 'Services': ['Breakfast available', 'Pool', 'Bar', 'Spa', '24/7 front desk', 'Air conditioning'], 'Property Offers': []}

Processing link 45
Extracted Hotel Name: INNSiDE by Meliá Bangkok Sukhumvit
Extracted Review Keyword: Wonderful
Number of services: 6
Extracted Hotel Data: {'Location': 'Bangkok, Thailand', 'Hotel Name': 'INNSi


Processing link 62
Extracted Hotel Name: The Siam
Extracted Review Keyword: Exceptional
Number of services: 6
Extracted Hotel Data: {'Location': 'Bangkok, Thailand', 'Hotel Name': 'The Siam', 'Stars': 5, 'Rating': '9.8', 'Review Keyword': 'Exceptional', 'Services': ['Spa', 'Pool', 'Bar', 'Breakfast included', '24/7 front desk', 'Parking included'], 'Property Offers': []}

Processing link 63
Extracted Hotel Name: Siam Kempinski Hotel Bangkok
Extracted Review Keyword: Exceptional
Number of services: 6
Extracted Hotel Data: {'Location': 'Bangkok, Thailand', 'Hotel Name': 'Siam Kempinski Hotel Bangkok', 'Stars': 5, 'Rating': '9.6', 'Review Keyword': 'Exceptional', 'Services': ['Pool', 'Breakfast available', 'Restaurant', 'Pet friendly', 'Bar', 'Room service'], 'Property Offers': []}

Processing link 64
Error extracting hotel name: Message: 
Stacktrace:
	GetHandleVerifier [0x012534A3+25059]
	(No symbol) [0x011DCEA4]
	(No symbol) [0x010BBEC3]
	(No symbol) [0x010FFD86]
	(No symbol) [0x010FFF

Extracted Hotel Name: Novotel Bangkok Platinum
Extracted Review Keyword: Excellent
Number of services: 6
Extracted Hotel Data: {'Location': 'Bangkok, Thailand', 'Hotel Name': 'Novotel Bangkok Platinum', 'Stars': 5, 'Rating': '8.6', 'Review Keyword': 'Excellent', 'Services': ['Breakfast available', 'Restaurant', 'Parking available', 'Pool', 'Bar', 'Spa'], 'Property Offers': []}

Processing link 70
Extracted Hotel Name: Lohas Residences Sukhumvit
Extracted Review Keyword: Very good
Number of services: 6
Extracted Hotel Data: {'Location': 'Bangkok, Thailand', 'Hotel Name': 'Lohas Residences Sukhumvit', 'Stars': 5, 'Rating': '8.0', 'Review Keyword': 'Very good', 'Services': ['Pool', 'Bar', 'Spa', 'Parking available', 'Gym', 'Room service'], 'Property Offers': []}

Processing link 71
Extracted Hotel Name: Centre Point Pratunam
Extracted Review Keyword: Excellent
Number of services: 6
Extracted Hotel Data: {'Location': 'Bangkok, Thailand', 'Hotel Name': 'Centre Point Pratunam', 'Stars': 5, '

...


Processing link 140
Extracted Hotel Name: GLOW Sukhumvit 5 by Centropolis
Extracted Review Keyword: Very good
Number of services: 6
Extracted Hotel Data: {'Location': 'Bangkok, Thailand', 'Hotel Name': 'GLOW Sukhumvit 5 by Centropolis', 'Stars': 4, 'Rating': '8.2', 'Review Keyword': 'Very good', 'Services': ['Pool', 'Bar', 'Room service', 'Breakfast available', 'Gym', 'Parking included'], 'Property Offers': []}

Processing link 141
Extracted Hotel Name: Amdaeng Bangkok Riverside Hotel
Extracted Review Keyword: Excellent
Number of services: 6
Extracted Hotel Data: {'Location': 'Bangkok, Thailand', 'Hotel Name': 'Amdaeng Bangkok Riverside Hotel', 'Stars': 4, 'Rating': '8.8', 'Review Keyword': 'Excellent', 'Services': ['Pool', 'Bar', 'Breakfast available', 'Laundry', 'Parking included', 'Room service'], 'Property Offers': []}

Processing link 142
Extracted Hotel Name: Miloft Sathorn hotel
Error extracting rating: Message: 
Stacktrace:
	GetHandleVerifier [0x012534A3+25059]
	(No symbol) [0


Processing link 153
Extracted Hotel Name: The Dawin Hotel
Extracted Review Keyword: Wonderful
Number of services: 6
Extracted Hotel Data: {'Location': 'Bangkok, Thailand', 'Hotel Name': 'The Dawin Hotel', 'Stars': 4, 'Rating': '9.0', 'Review Keyword': 'Wonderful', 'Services': ['Bar', 'Pool', 'Room service', 'Breakfast available', '24/7 front desk', 'Laundry'], 'Property Offers': []}

Processing link 154
Extracted Hotel Name: Thomson Residence Hotel
Extracted Review Keyword: Excellent
Number of services: 6
Extracted Hotel Data: {'Location': 'Bangkok, Thailand', 'Hotel Name': 'Thomson Residence Hotel', 'Stars': 4, 'Rating': '8.6', 'Review Keyword': 'Excellent', 'Services': ['Pool', 'Room service', 'Breakfast available', '24/7 front desk', 'Parking included', 'Gym'], 'Property Offers': []}

Processing link 155
Extracted Hotel Name: Green Point Residence Hotel
Extracted Review Keyword: Very good
Number of services: 6
Extracted Hotel Data: {'Location': 'Bangkok, Thailand', 'Hotel Name': 'G


Processing link 166
Extracted Hotel Name: Ibis Styles Bangkok Sukhumvit 50
Error extracting rating: Message: 
Stacktrace:
	GetHandleVerifier [0x012534A3+25059]
	(No symbol) [0x011DCEA4]
	(No symbol) [0x010BBEC3]
	(No symbol) [0x010FFD86]
	(No symbol) [0x010FFFCB]
	(No symbol) [0x0113D952]
	(No symbol) [0x01121F44]
	(No symbol) [0x0113B51E]
	(No symbol) [0x01121C96]
	(No symbol) [0x010F3FAC]
	(No symbol) [0x010F4F3D]
	GetHandleVerifier [0x01545613+3113811]
	GetHandleVerifier [0x0155A2DA+3199002]
	GetHandleVerifier [0x01552AB2+3168242]
	GetHandleVerifier [0x012F3310+680016]
	(No symbol) [0x011E57ED]
	(No symbol) [0x011E2A98]
	(No symbol) [0x011E2C35]
	(No symbol) [0x011D5890]
	BaseThreadInitThunk [0x7510FCC9+25]
	RtlGetAppContainerNamedObjectPath [0x7725809E+286]
	RtlGetAppContainerNamedObjectPath [0x7725806E+238]

Extracted Review Keyword: Good
Number of services: 6
Extracted Hotel Data: {'Location': 'Bangkok, Thailand', 'Hotel Name': 'Ibis Styles Bangkok Sukhumvit 50', 'Stars': 4, 'Ra

Extracted Hotel Name: Nanda Heritage Hotel
Extracted Review Keyword: Excellent
Number of services: 6
Extracted Hotel Data: {'Location': 'Bangkok, Thailand', 'Hotel Name': 'Nanda Heritage Hotel', 'Stars': 4, 'Rating': '8.6', 'Review Keyword': 'Excellent', 'Services': ['Bar', 'Pool', 'Breakfast available', 'Room service', 'Parking included', '24/7 front desk'], 'Property Offers': []}

Processing link 178
Extracted Hotel Name: T2 Residence Sathorn
Extracted Review Keyword: Wonderful
Number of services: 5
Extracted Hotel Data: {'Location': 'Bangkok, Thailand', 'Hotel Name': 'T2 Residence Sathorn', 'Stars': 4, 'Rating': '9.2', 'Review Keyword': 'Wonderful', 'Services': ['Pool', 'Housekeeping', 'Laundry', 'Air conditioning', 'Free WiFi'], 'Property Offers': []}

Processing link 179
Extracted Hotel Name: Aspira G Sukhumvit 33
Extracted Review Keyword: Very good
Number of services: 5
Extracted Hotel Data: {'Location': 'Bangkok, Thailand', 'Hotel Name': 'Aspira G Sukhumvit 33', 'Stars': 4, 'Rat


Processing link 191
Extracted Hotel Name: A-port
Extracted Review Keyword: Very good
Number of services: 6
Extracted Hotel Data: {'Location': 'Bangkok, Thailand', 'Hotel Name': 'A-port', 'Stars': 3, 'Rating': '8.2', 'Review Keyword': 'Very good', 'Services': ['Room service', 'Parking included', '24/7 front desk', 'Housekeeping', 'Air conditioning', 'Free WiFi'], 'Property Offers': []}

Processing link 192
Extracted Hotel Name: ibis Bangkok IMPACT
Extracted Review Keyword: Excellent
Number of services: 6
Extracted Hotel Data: {'Location': 'Bangkok, Thailand', 'Hotel Name': 'ibis Bangkok IMPACT', 'Stars': 3, 'Rating': '8.8', 'Review Keyword': 'Excellent', 'Services': ['Bar', 'Breakfast available', '24/7 front desk', 'Laundry', 'Housekeeping', 'Parking included'], 'Property Offers': []}

Processing link 193
Extracted Hotel Name: Thong Ta Resort And Spa
Error extracting rating: Message: 
Stacktrace:
	GetHandleVerifier [0x012534A3+25059]
	(No symbol) [0x011DCEA4]
	(No symbol) [0x010BBEC3]
